<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-assets/phoenix/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">Evaluating an Agent</h1>



## Install Dependencies, Import Libraries, Set API Keys

In [1]:
!pip install -q openai arize-phoenix openinference-instrumentation-openai python-dotenv duckdb
!pip install -qU git+https://github.com/Arize-AI/openinference.git@xander/oitracer#subdirectory=python/openinference-instrumentation


In [30]:
import dotenv

dotenv.load_dotenv()

from openai import OpenAI
from phoenix.otel import register
from opentelemetry.trace import StatusCode
from openinference.instrumentation.openai import OpenAIInstrumentor
from openinference.semconv.trace import SpanAttributes
import duckdb
import json
import os
from getpass import getpass
import pandas as pd
from pydantic import BaseModel, Field
from IPython.display import Markdown
from tqdm import tqdm

import json
from typing import Any, Dict

from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace.export import SimpleSpanProcessor
from opentelemetry.trace import Status, StatusCode, set_tracer_provider

from openinference.instrumentation.config import (
    OpenInferenceTracerProvider,
    suppress_tracing,
)
from openinference.semconv.resource import ResourceAttributes
import phoenix as px


# Enable Phoenix Tracing

In [3]:
if os.getenv("OPENAI_API_KEY") is None:
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

client = OpenAI()
model = "gpt-4o-mini"
project_name = "github-1-15-2025-live"

In [4]:

# This will be replaced with the Phoenix register() call
endpoint = "http://127.0.0.1:6006/v1/traces"
resource = Resource(attributes={ResourceAttributes.PROJECT_NAME: project_name})
tracer_provider = OpenInferenceTracerProvider(resource=resource)
tracer_provider.add_span_processor(SimpleSpanProcessor(OTLPSpanExporter(endpoint)))
set_tracer_provider(tracer_provider)
tracer = tracer_provider.get_tracer(__name__)

session = px.launch_app()

# Initialize OpenAI instrumentation
OpenAIInstrumentor().instrument(tracer_provider=tracer_provider)

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


## Prepare dataset

In [5]:
store_sales_df = pd.read_parquet("https://storage.googleapis.com/arize-phoenix-assets/datasets/unstructured/llm/llama-index/Store_Sales_Price_Elasticity_Promotions_Data.parquet")
store_sales_df.head()

,Store_Number,SKU_Coded,Product_Class_Code,Sold_Date,Qty_Sold,Total_Sale_Value,On_Promo
0,1320,6172800,22875,2021-11-02,3,56.849998,0
1,2310,6172800,22875,2021-11-03,1,18.950001,0
2,3080,6172800,22875,2021-11-03,1,18.950001,0
3,2310,6172800,22875,2021-11-06,1,18.950001,0
4,4840,6172800,22875,2021-11-07,1,18.950001,0


## Define the tools

### Tool 1: Database Lookup

In [6]:
SQL_GENERATION_PROMPT = """
Generate an SQL query based on a prompt. Do not reply with anything besides the SQL query.
The prompt is: {prompt}

The available columns are: {columns}
The table name is: {table_name}
"""

def generate_sql_query(prompt: str, columns: list, table_name: str) -> str:
    """Generate an SQL query based on a prompt"""
    formatted_prompt = SQL_GENERATION_PROMPT.format(prompt=prompt, columns=columns, table_name=table_name)

    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": formatted_prompt}],
    )
    
    return response.choices[0].message.content

def lookup_sales_data(prompt: str) -> str:
    """Implementation of sales data lookup from parquet file using SQL"""
    try:
        table_name = "sales"
        # Read the parquet file into a DuckDB table
        duckdb.sql(f"CREATE TABLE IF NOT EXISTS {table_name} AS SELECT * FROM store_sales_df")
        
        sql_query = generate_sql_query(prompt, store_sales_df.columns, table_name)
        sql_query = sql_query.strip()
        sql_query = sql_query.replace("```sql", "").replace("```", "")
        
        with tracer.start_as_current_span("execute_sql_query") as span:
            span.set_attribute(SpanAttributes.OPENINFERENCE_SPAN_KIND, "CHAIN")
            span.set_attribute(SpanAttributes.INPUT_VALUE, sql_query)
            span.set_attribute(SpanAttributes.INPUT_MIME_TYPE, "application/json")
            
            # Execute the SQL query
            result = duckdb.sql(sql_query).df()
            span.set_attribute(SpanAttributes.OUTPUT_VALUE, str(result))
            span.set_status(StatusCode.OK)
        return result.to_string()
    except Exception as e:
        return f"Error accessing data: {str(e)}"

In [7]:
# example_data = lookup_sales_data("Show me all the sales for store 1320 on November 1st, 2021")
# example_data

### Tool 2: Data Visualization

In [8]:
class VisualizationConfig(BaseModel):
    chart_type: str = Field(..., description="Type of chart to generate")
    x_axis: str = Field(..., description="Name of the x-axis column")
    y_axis: str = Field(..., description="Name of the y-axis column")
    title: str = Field(..., description="Title of the chart")

def extract_chart_config(data: str, visualization_goal: str) -> dict:
    """Generate chart visualization configuration
    
    Args:
        data: String containing the data to visualize
        visualization_goal: Description of what the visualization should show
        
    Returns:
        Dictionary containing line chart configuration
    """
    prompt = f"""Generate a chart configuration based on this data: {data}
    The goal is to show: {visualization_goal}"""
    
    response = client.beta.chat.completions.parse(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        response_format=VisualizationConfig,
    )
    
    try:
        # Extract axis and title info from response
        content = response.choices[0].message.content
        
        # Return structured chart config
        return {
            "chart_type": content.chart_type,
            "x_axis": content.x_axis,
            "y_axis": content.y_axis,
            "title": content.title,
            "data": data
        }
    except Exception:
        return {
            "chart_type": "line", 
            "x_axis": "date",
            "y_axis": "value",
            "title": visualization_goal,
            "data": data
        }
        
def create_chart(config: VisualizationConfig) -> str:
    """Create a chart based on the configuration"""
    prompt = f"""Write python code to create a chart based on the following configuration.
    Only return the code, no other text.
    config: {config}"""
    
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
    )
    
    code = response.choices[0].message.content
    code = code.replace("```python", "").replace("```", "")
    code = code.strip()
    
    return code

def generate_visualization(data: str, visualization_goal: str) -> str:
    """Generate a visualization based on the data and goal"""
    config = extract_chart_config(data, visualization_goal)
    code = create_chart(config)
    return code

In [9]:
# code = generate_visualization(example_data, "A line chart of sales over each day in november.")

In [10]:
def run_python_code(code: str) -> str:
    """Execute Python code in a restricted environment"""
    # Create restricted globals/locals dictionaries with plotting libraries
    restricted_globals = {
        '__builtins__': {
            'print': print,
            'len': len,
            'range': range,
            'sum': sum,
            'min': min,
            'max': max,
            'int': int, 
            'float': float,
            'str': str,
            'list': list,
            'dict': dict,
            'tuple': tuple,
            'set': set,
            'round': round,
            '__import__': __import__,
            'json': __import__('json')
        },
        'plt': __import__('matplotlib.pyplot'),
        'pd': __import__('pandas'),
        'np': __import__('numpy'),
        'sns': __import__('seaborn')
    }
    
    try:
        # Execute code in restricted environment
        exec_locals = {}
        exec(code, restricted_globals, exec_locals)
        
        # Capture any printed output or return the plot
        output = exec_locals.get('__builtins__', {}).get('_', '')
        if 'plt' in exec_locals:
            return exec_locals['plt']
        
        # Try to parse output as JSON before returning
        return "Code executed successfully"
        
    except Exception as e:
        return f"Error executing code: {str(e)}"

### Tool 3: Data Analysis

In [11]:
def analyze_sales_data(prompt: str, data: str) -> str:
    """Implementation of AI-powered sales data analysis"""
    # Construct prompt based on analysis type and data subset
    prompt = f"""Analyze the following data: {data}
    Your job is to answer the following question: {prompt}"""
    
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
    )
    
    analysis = response.choices[0].message.content
    return analysis if analysis else "No analysis could be generated"

In [12]:
# analysis = analyze_sales_data("What is the most popular product SKU?", example_data)
# analysis


### Tool Schema:

In [13]:
# Define tools/functions that can be called by the model
tools = [
    {
        "type": "function",
        "function": {
            "name": "lookup_sales_data",
            "description": "Look up data from Store Sales Price Elasticity Promotions dataset",
            "parameters": {
                "type": "object",
                "properties": {
                    "prompt": {"type": "string", "description": "The unchanged prompt that the user provided."}
                },
                "required": ["prompt"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "analyze_sales_data", 
            "description": "Analyze sales data to extract insights",
            "parameters": {
                "type": "object",
                "properties": {
                    "data": {"type": "string", "description": "The lookup_sales_data tool's output."},
                    "prompt": {"type": "string", "description": "The unchanged prompt that the user provided."}
                },
                "required": ["data", "prompt"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "generate_visualization",
            "description": "Generate Python code to create data visualizations",
            "parameters": {
                "type": "object", 
                "properties": {
                    "data": {"type": "string", "description": "The lookup_sales_data tool's output."},
                    "visualization_goal": {"type": "string", "description": "The goal of the visualization."}
                },
                "required": ["data", "visualization_goal"]
            }
        }
    }
]

# Dictionary mapping function names to their implementations
tool_implementations = {
    "lookup_sales_data": lookup_sales_data,
    "analyze_sales_data": analyze_sales_data, 
    "generate_visualization": generate_visualization
}


## Agent logic

In [14]:
def handle_tool_calls(tool_calls, messages):
    for tool_call in tool_calls:
        with tracer.start_as_current_span(f"tool_call_{tool_call.function.name}") as span:
            span.set_attribute(SpanAttributes.OPENINFERENCE_SPAN_KIND, "TOOL")
            span.set_attribute(SpanAttributes.INPUT_VALUE, str(tool_call))
            span.set_attribute(SpanAttributes.INPUT_MIME_TYPE, "application/json")
            span.set_attribute(SpanAttributes.TOOL_NAME, tool_call.function.name)
            span.set_attribute(SpanAttributes.TOOL_PARAMETERS, str(tool_call.function.arguments))
            
            function = tool_implementations[tool_call.function.name]
            function_args = json.loads(tool_call.function.arguments)
            result = function(**function_args)
            
            span.set_attribute(SpanAttributes.OUTPUT_VALUE, str(result))
            
            messages.append({"role": "tool", "content": result, "tool_call_id": tool_call.id})
    return messages


In [15]:
def start_main_span(messages):
    print("Starting main span with messages:", messages)
    
    with tracer.start_as_current_span("AgentRun") as span:
        span.set_attribute(SpanAttributes.OPENINFERENCE_SPAN_KIND, "AGENT")
        span.set_attribute(SpanAttributes.INPUT_VALUE, str(messages))
        span.set_attribute(SpanAttributes.INPUT_MIME_TYPE, "application/json")
        ret = run_agent(messages)
        print("Main span completed with return value:", ret)
        span.set_attribute(SpanAttributes.OUTPUT_VALUE, str(ret))
        span.set_status(StatusCode.OK)
        return ret

def run_agent(messages):
    print("Running agent with messages:", messages)
    if isinstance(messages, str):
        messages = [{"role": "user", "content": messages}]
        print("Converted string message to list format")
    
    # Check and add system prompt if needed
    if not any(
            isinstance(message, dict) and message.get("role") == "system" for message in messages
        ):
            system_prompt = {"role": "system", "content": "You are a helpful assistant that can answer questions about the Store Sales Price Elasticity Promotions dataset."}
            messages.append(system_prompt)
            print("Added system prompt to messages")

    while True:
        # Router call span
        print("Starting router call span")
        with tracer.start_as_current_span(
            "router_call",
            openinference_span_kind="chain",
        ) as span:
            span.set_input(value=json.dumps(messages), mime_type="application/json")
            
            response = client.chat.completions.create(
                model=model,
                messages=messages,
                tools=tools,
            )

            messages.append(response.choices[0].message.model_dump())
            tool_calls = response.choices[0].message.tool_calls
            print("Received response with tool calls:", bool(tool_calls))
            span.set_status(StatusCode.OK)
            
            if tool_calls:
                # Tool calls span
                print("Processing tool calls")
                tool_calls = response.choices[0].message.tool_calls

                with tracer.start_as_current_span(
                    "tool_calls",
                    openinference_span_kind="chain",
                ) as tool_span:
                    tool_span.set_input(value=str(tool_calls))
                    messages = handle_tool_calls(tool_calls, messages)
                    tool_span.set_output(value=json.dumps(messages), mime_type="application/json")
                    tool_span.set_status(StatusCode.OK)
                span.set_output(value=str(response.choices[0].message.tool_calls), mime_type="application/json")
            else:
                print("No tool calls, returning final response")
                span.set_output(value=response.choices[0].message.content)

                return response.choices[0].message.content


## Run the agent

In [16]:
ret = start_main_span([{"role": "user", "content": "Which stores did the best in 2021"}])
print(Markdown(ret))

Starting main span with messages: [{'role': 'user', 'content': 'Which stores did the best in 2021'}]
Running agent with messages: [{'role': 'user', 'content': 'Which stores did the best in 2021'}]
Added system prompt to messages
Starting router call span
Received response with tool calls: True
Processing tool calls
Starting router call span
Received response with tool calls: False
No tool calls, returning final response
Main span completed with return value: In 2021, the stores that performed the best in terms of total sales were:

1. Store Number 2970: $84,454.33
2. Store Number 3300: $63,205.33
3. Store Number 1650: $62,152.43
4. Store Number 1540: $58,777.02
5. Store Number 1210: $55,435.62

These stores had the highest total sales figures for the year.
<IPython.core.display.Markdown object>


In [ ]:
agent_questions = [
    "What was the most popular product SKU?",
    "What was the total revenue across all stores?",
    "Which store had the highest sales volume?",
    "Create a bar chart showing total sales by store",
    "What percentage of items were sold on promotion?",
    "Plot daily sales volume over time", 
    "What was the average transaction value?",
    "Create a box plot of transaction values",
    "Which products were frequently purchased together?",
    "Plot a line graph showing the sales trend over time with a 7-day moving average"
]

for question in tqdm(agent_questions, desc="Processing questions"):
    try:
        ret = start_main_span([{"role": "user", "content": question}])
    except Exception as e:
        print(f"Error processing question: {question}")
        print(e)
        continue

# Evaluating the agent

In [18]:
OpenAIInstrumentor().uninstrument()

In [21]:
import phoenix as px
from phoenix.evals import (
    TOOL_CALLING_PROMPT_TEMPLATE, 
    llm_classify,
    OpenAIModel
)
from phoenix.trace import SpanEvaluations
from phoenix.experiments import run_experiment
import phoenix as px
from phoenix.trace.dsl import SpanQuery

import nest_asyncio
nest_asyncio.apply()

In [20]:
px_client = px.Client()
eval_model = OpenAIModel(model="gpt-4o-mini")

## Function Calling Evals using LLM as a Judge

In [22]:
query = SpanQuery().where(
    # Filter for the `RETRIEVER` span kind.
    # The filter condition is a string of valid Python boolean expression.
    "span_kind == 'LLM'",
).select(
    # Extract the span attribute `input.value` which contains the query for the
    # retriever. Rename it as the `input` column in the output dataframe.
    question="input.value",
    output_messages="llm.tools"
    
)

# The Phoenix Client can take this query and return the dataframe.
tool_calls_df = px.Client().query_spans(query, project_name=project_name, timeout=None)
tool_calls_df = tool_calls_df.dropna(subset=["output_messages"])
tool_calls_df["tool_call"] = tool_calls_df["output_messages"]

tool_calls_df.head()

,question,output_messages,tool_call
context.span_id,,,
d8d048cfadc1b8fb,"{""messages"": [{""role"": ""user"", ""content"": ""Whi...","[{'tool': {'json_schema': '{""type"": ""function""...","[{'tool': {'json_schema': '{""type"": ""function""..."
5eeb77eb2c180656,"{""messages"": [{""role"": ""user"", ""content"": ""Whi...","[{'tool': {'json_schema': '{""type"": ""function""...","[{'tool': {'json_schema': '{""type"": ""function""..."
243ffb56898e032c,"{""messages"": [{""role"": ""user"", ""content"": ""Wha...","[{'tool': {'json_schema': '{""type"": ""function""...","[{'tool': {'json_schema': '{""type"": ""function""..."


In [23]:
tool_call_eval = llm_classify(
    dataframe = tool_calls_df,
    template = TOOL_CALLING_PROMPT_TEMPLATE.template.replace("{tool_definitions}", "generate_visualization, lookup_sales_data, analyze_sales_data, run_python_code"),
    rails = ['correct', 'incorrect'],
    model=eval_model,
    provide_explanation=True
)

tool_call_eval['score'] = tool_call_eval.apply(lambda x: 1 if x['label']=='correct' else 0, axis=1)

tool_call_eval.head()

llm_classify |          | 0/3 (0.0%) | ⏳ 00:00<? | ?it/s

,label,explanation,exceptions,execution_status,execution_seconds,score
context.span_id,,,,,,
d8d048cfadc1b8fb,correct,The tools called include 'lookup_sales_data' t...,[],COMPLETED,1.931483,1
5eeb77eb2c180656,correct,The tool call to 'lookup_sales_data' is approp...,[],COMPLETED,2.468532,1
243ffb56898e032c,incorrect,The chosen tools include 'lookup_sales_data' t...,[],COMPLETED,2.350198,0


In [24]:
px.Client().log_evaluations(
    SpanEvaluations(eval_name="Tool Calling Eval", dataframe=tool_call_eval),
)

### Function Calling Evals using Ground Truth

In [25]:
agent_tool_responses = {
    "What was the most popular product SKU?": "lookup_sales_data, analyze_sales_data",
    "What was the total revenue across all stores?": "lookup_sales_data, analyze_sales_data",
    "Which store had the highest sales volume?": "lookup_sales_data, analyze_sales_data",
    "Create a bar chart showing total sales by store": "generate_visualization, lookup_sales_data, run_python_code",
    "What percentage of items were sold on promotion?": "lookup_sales_data, analyze_sales_data",
    "Plot daily sales volume over time": "generate_visualization, lookup_sales_data, run_python_code", 
    "What was the average transaction value?": "lookup_sales_data, analyze_sales_data",
    "Create a box plot of transaction values": "generate_visualization, lookup_sales_data, run_python_code",
    "Which products were frequently purchased together?": "lookup_sales_data, analyze_sales_data",
    "Plot a line graph showing the sales trend over time with a 7-day moving average": "generate_visualization, lookup_sales_data, run_python_code"
}


tool_calling_df = pd.DataFrame(agent_tool_responses.items(), columns=["question", "tool_calls"])
dataset = px_client.upload_dataset(dataframe=tool_calling_df, dataset_name="tool_calling_ground_truth", input_keys=["question"], output_keys=["tool_calls"])

📤 Uploading dataset...
💾 Examples uploaded: http://localhost:6006/datasets/RGF0YXNldDox/examples
🗄️ Dataset version ID: RGF0YXNldFZlcnNpb246MQ==


In [26]:
from phoenix.experiments.types import Example

def run_router_step(example: Example) -> str:
    messages = [{"role": "system", "content": "You are a helpful assistant that can answer questions about the Store Sales Price Elasticity Promotions dataset."}]
    messages.append({"role": "user", "content": example.input.get("question")})
    
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        tools=tools,
    )
    tool_calls = []
    for tool_call in response.choices[0].message.tool_calls:
        tool_calls.append(tool_call.function.name)
    return tool_calls


In [27]:
def tools_match(expected: str, output: str) -> bool:
    expected_tools = expected.get("tool_calls").split(", ")
    return expected_tools == output


In [28]:
experiment = run_experiment(dataset,
                            run_router_step,
                            evaluators=[tools_match],
                            experiment_name="Tool Calling Eval",
                            experiment_description="Evaluating the tool calling step of the agent")


🧪 Experiment started.
📺 View dataset experiments: http://localhost:6006/datasets/RGF0YXNldDox/experiments
🔗 View this experiment: http://localhost:6006/datasets/RGF0YXNldDox/compare?experimentId=RXhwZXJpbWVudDox


running tasks |          | 0/10 (0.0%) | ⏳ 00:00<? | ?it/s

✅ Task runs completed.
🧠 Evaluation started.


running experiment evaluations |          | 0/10 (0.0%) | ⏳ 00:00<? | ?it/s


🔗 View this experiment: http://localhost:6006/datasets/RGF0YXNldDox/compare?experimentId=RXhwZXJpbWVudDox

Experiment Summary (01/16/25 06:00 PM -0800)
--------------------------------------------
| evaluator   |   n |   n_scores |   avg_score |   n_labels | top_2_labels   |
|:------------|----:|-----------:|------------:|-----------:|:---------------|
| tools_match |  10 |         10 |           0 |         10 | {'False': 10}  |

Tasks Summary (01/16/25 06:00 PM -0800)
---------------------------------------
|   n_examples |   n_runs |   n_errors |
|-------------:|---------:|-----------:|
|           10 |       10 |          0 |


## Tool Evals

### Evaluating our SQL generation tool

In [31]:
# This step will be replaced by a human annotated set of ground truth data, instead of generated examples

db_lookup_questions = [
    "What was the most popular product SKU?",
    "Which store had the highest total sales value?", 
    "How many items were sold on promotion?",
    "What was the average quantity sold per transaction?",
    "Which product class code generated the most revenue?",
    "What day of the week had the highest sales volume?",
    "How many unique stores made sales?",
    "What was the highest single transaction value?",
    "Which products were frequently sold together?",
    "What's the trend in sales over time?"
]

expected_results = []

for question in tqdm(db_lookup_questions, desc="Processing SQL lookup questions"):
    try:
        with suppress_tracing():
            expected_results.append(lookup_sales_data(question))
    except Exception as e:
        print(f"Error processing question: {question}")
        print(e)
        db_lookup_questions.remove(question)

# Create a DataFrame with the questions
questions_df = pd.DataFrame({
    'question': db_lookup_questions,
    'expected_result': expected_results
})

display(questions_df)












Processing SQL lookup questions: 100%|██████████| 10/10 [00:12<00:00,  1.28s/it]


,question,expected_result
0,What was the most popular product SKU?,SKU_Coded Total_Quantity_Sold\n0 620070...
1,Which store had the highest total sales value?,Store_Number Total_Sales_Value\n0 ...
2,How many items were sold on promotion?,Total_Items_Sold_On_Promotion\n0 ...
3,What was the average quantity sold per transac...,Average_Quantity_Sold_Per_Transaction\n0 ...
4,Which product class code generated the most re...,Product_Class_Code Total_Revenue\n0 ...
5,What day of the week had the highest sales vol...,Day_Of_Week Total_Sales_Volume\n0 ...
6,How many unique stores made sales?,unique_stores\n0 35
7,What was the highest single transaction value?,Highest_Single_Transaction_Value\n0 ...
8,Which products were frequently sold together?,Product_A Product_B Frequency\n0 ...
9,What's the trend in sales over time?,Sales_Month Total_Quantity_Sold Total_Sal...


In [32]:
dataset = px_client.upload_dataset(dataframe=questions_df, dataset_name="sales_db_lookup_questions", input_keys=["question"], output_keys=["expected_result"])

📤 Uploading dataset...
💾 Examples uploaded: http://localhost:6006/datasets/RGF0YXNldDoy/examples
🗄️ Dataset version ID: RGF0YXNldFZlcnNpb246Mg==


In [33]:
def run_sql_query(example: Example) -> str:
    with suppress_tracing():
        return lookup_sales_data(example.input.get("question"))

In [34]:
def evaluate_sql_result(output: str, expected: str) -> bool:    
    # Extract just the numbers from both strings
    result_nums = ''.join(filter(str.isdigit, output))
    expected_nums = ''.join(filter(str.isdigit, expected.get("expected_result")))
    return result_nums == expected_nums

In [35]:
experiment = run_experiment(dataset,
                            run_sql_query,
                            evaluators=[evaluate_sql_result],
                            experiment_name="SQL Query Eval",
                            experiment_description="Evaluating the SQL query generation step of the agent")

🧪 Experiment started.
📺 View dataset experiments: http://localhost:6006/datasets/RGF0YXNldDoy/experiments
🔗 View this experiment: http://localhost:6006/datasets/RGF0YXNldDoy/compare?experimentId=RXhwZXJpbWVudDoy


running tasks |          | 0/10 (0.0%) | ⏳ 00:00<? | ?it/s

✅ Task runs completed.
🧠 Evaluation started.


running experiment evaluations |          | 0/10 (0.0%) | ⏳ 00:00<? | ?it/s


🔗 View this experiment: http://localhost:6006/datasets/RGF0YXNldDoy/compare?experimentId=RXhwZXJpbWVudDoy

Experiment Summary (01/16/25 06:02 PM -0800)
--------------------------------------------
| evaluator           |   n |   n_scores |   avg_score |   n_labels | top_2_labels            |
|:--------------------|----:|-----------:|------------:|-----------:|:------------------------|
| evaluate_sql_result |  10 |         10 |         0.8 |         10 | {'True': 8, 'False': 2} |

Tasks Summary (01/16/25 06:02 PM -0800)
---------------------------------------
|   n_examples |   n_runs |   n_errors |
|-------------:|---------:|-----------:|
|           10 |       10 |          0 |


### Evaluating our Python code generation tool

In [36]:
# Replace this with a human annotated set of ground truth data, instead of generated examples

code_generation_questions = [
    "Create a bar chart showing total sales by store",
    "Plot daily sales volume over time", 
    "Plot a line graph showing the sales trend over time with a 7-day moving average",
    "Create a histogram of quantities sold per transaction",
    "Generate a pie chart showing sales distribution across product classes",
    "Create a stacked bar chart showing promotional vs non-promotional sales by store",
    "Generate a heatmap of sales by day of week and store number",
    "Plot a line chart comparing sales trends between top 5 stores"
]

example_data = []
chart_configs = []
for question in tqdm(code_generation_questions[:], desc="Processing code generation questions"):
    try:
        with suppress_tracing():
            example_data.append(lookup_sales_data(question))
            chart_configs.append(json.dumps(extract_chart_config(example_data[-1], question)))
    except Exception as e:
        print(f"Error processing question: {question}")
        print(e)
        code_generation_questions.remove(question)

code_generation_df = pd.DataFrame({
    'question': code_generation_questions,
    'example_data': example_data,
    'chart_configs': chart_configs
})

dataset = px_client.upload_dataset(dataframe=code_generation_df, dataset_name="code_generation_questions", input_keys=["question", "example_data", "chart_configs"])




























Processing code generation questions: 100%|██████████| 8/8 [00:20<00:00,  2.51s/it]

📤 Uploading dataset...
💾 Examples uploaded: http://localhost:6006/datasets/RGF0YXNldDoz/examples
🗄️ Dataset version ID: RGF0YXNldFZlcnNpb246Mw==


In [46]:
def run_code_generation(example: Example) -> str:
    with suppress_tracing():
        chart_config = extract_chart_config(data=example.input.get("example_data"), visualization_goal=example.input.get("question"))
        code = generate_visualization(visualization_goal=example.input.get("question"), 
                                    data=example.input.get("example_data"))
    
    return {"code": code, "chart_config": chart_config}

In [38]:
def code_is_runnable(output: str) -> bool:
    """Check if the code is runnable"""
    output = output.get("code")
    output = output.strip()
    output = output.replace("```python", "").replace("```", "")
    try:
        exec(output)
        return True
    except Exception as e:
        return False


In [39]:
def evaluate_chart_config(output: str, expected: str) -> bool:
    return output.get("chart_config") == expected.get("chart_config")

In [47]:
experiment = run_experiment(dataset,
                            run_code_generation,
                            evaluators=[code_is_runnable, evaluate_chart_config],
                            experiment_name="Code Generation Eval",
                            experiment_description="Evaluating the code generation step of the agent")

# Evaluating the agent path and convergence